# Анализ базы данных книжного сервиса

В данном проекте необходимо проанализировать базу данных книжного сервиса.

**Цель проекта**

Ad-hoc анализ для формирования ценностного предложение для нового продукта.

*Входные данные* - `база данных книжного сервиса с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг`

**Описание данных**



**Таблица books**

Содержит данные о книгах:

 - `book_id` — идентификатор книги;

 - `author_id` — идентификатор автора;

 - `title` — название книги;

 - `num_pages` — количество страниц;

 - `publication_date` — дата публикации книги;

 - `publisher_id` — идентификатор издателя.


**Таблица authors**

Содержит данные об авторах:

 - `author_id` — идентификатор автора;

 - `author` — имя автора.


**Таблица publishers**

Содержит данные об издательствах:

 - `publisher_id` — идентификатор издательства;

 - `publisher` — название издательства.


**Таблица ratings**

Содержит данные о пользовательских оценках книг:

 - `rating_id` — идентификатор оценки;

 - `book_id` — идентификатор книги;

 - `username` — имя пользователя, оставившего оценку;

 - `rating` — оценка книги.


**Таблица reviews**

Содержит данные о пользовательских обзорах:

 - `review_id` — идентификатор обзора;

 - `book_id` — идентификатор книги;

 - `username` — имя автора обзора;

 - `text` — текст обзора.

## Подключение к базе

In [ ]:
#импортируем необходимые библиотеки
import pandas as pd
import sqlalchemy as sa

# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
  with engine.connect() as con:
    return pd.read_sql(sql=sa.text(query), con = con)

In [ ]:
query = '''SELECT *
           FROM books
           LIMIT 5'''

get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [ ]:
query = '''SELECT MAX(publication_date)
           FROM books'''

get_sql_data(query)

,max
0,2020-03-31


## Анализ данных

### Количество книг, вышедших после 1 января 2000 года

In [ ]:
query = """SELECT COUNT(book_id)
           FROM books
           WHERE publication_date::date >= '2000-01-01'"""

get_sql_data(query)

,count
0,821


**Вывод**

Среди авторов, представленных в книжном сервисе с 1 января 2000 года по 31 марта 2020 года вышла 821 книга.

### Количество обзоров и средний рейтинг для каждой книги

In [ ]:
query = """SELECT title,
                  COUNT(DISTINCT review_id) cnt_reviews,
                  AVG(rating) avg_rating
           FROM books b
           LEFT JOIN reviews rev ON rev.book_id = b.book_id
           LEFT JOIN ratings rat ON b.book_id = rat.book_id
           GROUP BY title"""

get_sql_data(query)

,title,cnt_reviews,avg_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
...,...,...,...
994,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
995,Xenocide (Ender's Saga #3),3,3.400000
996,Year of Wonders,4,3.200000
997,You Suck (A Love Story #2),2,4.500000


In [ ]:
query = '''SELECT COUNT(DISTINCT rev.review_id) cnt_reviews,
                  AVG(rating) avg_rating
           FROM reviews rev
           LEFT JOIN ratings rat ON rev.book_id = rat.book_id'''

get_sql_data(query)

,cnt_reviews,avg_rating
0,2793,3.936791


**Вывод**

Общее количество обзоров на книги в сервисе составило 2 793, а средний рейтинг всех книг - 3,94.

### Издательство, выпустившее наибольшее количество книг

In [ ]:
query = '''SELECT publisher,
                  COUNT(b.book_id)
           FROM publishers p
           LEFT JOIN books b ON p.publisher_id = b.publisher_id
           WHERE num_pages > 50
           GROUP BY publisher
           ORDER BY COUNT(book_id) DESC
           LIMIT 1'''

get_sql_data(query)

,publisher,count
0,Penguin Books,42


**Вывод**

Издательство, которое выпустило наибольшее число книг является Penguin Books. Учитывались книги в которых больше 50 страниц, дабы исключить попадение брошюр при анализе.

### Автор с самой высокой средней оценкой книг

In [ ]:
query = '''SELECT a.author
           FROM authors a
           LEFT JOIN books b ON a.author_id = b.author_id
           LEFT JOIN ratings rat ON b.book_id = rat.book_id
           WHERE b.book_id IN (SELECT book_id
                               FROM (SELECT b.book_id,
                                            COUNT(rating) cnt_rating
                                     FROM books b
                                     LEFT JOIN ratings rat ON b.book_id = rat.book_id
                                     GROUP BY b.book_id) a
                                     WHERE cnt_rating >= 50)
           GROUP BY a.author
           ORDER BY AVG(rat.rating) DESC
           LIMIT 1'''

get_sql_data(query)

,author
0,J.K. Rowling/Mary GrandPré


**Вывод**

Автором с самой высокой средней оценкой в сервисе является J.K. Rowling/Mary GrandPré. Из анализа исключили авторов, у которых меньше 50 оценок.

### Среднее количество обзоров от пользователей

In [ ]:
query = '''SELECT AVG(cnt_review)
           FROM (SELECT COUNT(review_id) cnt_review
                 FROM reviews
                 WHERE username IN (SELECT username
                                    FROM ratings
                                    GROUP BY username
                                    HAVING COUNT(rating) > 48)
                 GROUP BY username) a'''

get_sql_data(query)

,avg
0,24.0


**Вывод**

Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составило 24.

# Общий вывод

Задачей проетка был ad-hoc анализ базы данных.

**Этапы проекта**

1. Подключение к базе


2. Анализ данных
 - Количество книг, вышедших после 1 января 2000 года
 - Количество обзоров и средний рейтинг для каждой книги
 - Издательство, выпустившее наибольшее количество книг
 - Автор с самой высокой средней оценкой книг
 - Среднее количество обзоров от пользователей

**Результаты полученные в ходе исследования**

- С 1 января 2000 года по 31 марта 2020 года вышла 821 книга
- Общее количество обзоров составило 2 793, средний рейтинг всех книг - 3,94
- Penguin Books - издательство, которое выпустило наибольшее число книг
- J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой
- Среднее количество обзоров от пользователей составило 24